## CBF

In [409]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pickle
import re
import nltk
from nltk.tokenize import word_tokenize

In [410]:
news_df = pd.read_csv('/content/news.csv')
rec_items_df = pd.read_csv('/content/rec_items.csv')
rec_feedback_df = pd.read_csv('/content/rec_feedback.csv')
users_df = pd.read_csv('/content/users.csv')
users_df = pd.read_csv('/content/users.csv')

In [411]:
news_df.head()

,id,title,description,published_date,breaking_news,blob_image,source_url,created_at,updated_at,published_at,created_by_id,updated_by_id,shares,comment_count,type
0,23996,අද ඩොලරයේ අගය,ශ්‍රී ලංකා මහ බැංකුව විසින් අද (13) දින නිකුත්...,2025-02-13 12:10:32.919,False,NaN,https://www.hirunews.lk/396721/%E0%B6%85%E0%B6...,2025-02-13 09:43:07.843,2025-02-13 09:43:12.403,2025-02-13 09:43:12.131,7.0,7.0,0,0,News
1,23995,‘ක්ලීන් ශ‍්‍රී ලංකා ලිඛිතව තියෙනවා.’ මාලිමා මන...,‘ක්‍ලීන් ශ්‍රී ලංකා’ වැඩපිළිවෙල යනු කුමක්දැයි ...,2025-02-13 09:30:00,False,NaN,https://lankacnews.com/%e0%b6%9a%e0%b7%8a%e0%b...,2025-02-13 09:42:37.114,2025-02-13 09:42:47.003,NaN,NaN,NaN,0,0,News
2,23994,පාපන්දු ගෝල කණුවක් කඩා වැටී පාසැල් සිසුවෙකු ජී...,පාසැල් ක්‍රීඩාගංනයක තිබූ පාපන්දු ගෝල කණුවක් කඩ...,2025-02-13 11:10:47.133,False,NaN,https://www.hirunews.lk/396719/%E0%B6%B4%E0%B7...,2025-02-13 09:33:20.185,2025-02-13 09:33:23.796,2025-02-13 09:33:23.747,7.0,7.0,0,0,News
3,23993,සුජීව සේනසිංහගේ මූලික අයිතිවාසිකම් පෙත්සම විභා...,තමන්ට එරෙහිව අපරාධ පරීක්ෂණ දෙපාර්තමේන්තුව විසි...,2025-02-13 10:10:29.598,False,NaN,https://www.hirunews.lk/396716/%E0%B7%83%E0%B7...,2025-02-13 09:31:55.442,2025-02-13 09:31:59.501,2025-02-13 09:31:59.41,7.0,7.0,0,0,News
4,23992,නීති විරෝධී ධීවර දැල් දෙසීය පනහක් නීතියේ රැහැනට,"ශ්‍රී ලංකා නාවික හමුදාව, කිලිනොච්චිය මුද්දලම්ප...",2025-02-13 10:58:56.639,False,NaN,https://www.dinamina.lk/2025/02/13/lawnorder/1...,2025-02-13 09:29:23.368,2025-02-13 09:29:26.299,2025-02-13 09:29:26.258,7.0,7.0,0,0,News


In [412]:
news_df.dtypes

,0
id,int64
title,object
description,object
published_date,object
breaking_news,bool
blob_image,float64
source_url,object
created_at,object
updated_at,object
published_at,object


In [413]:
print(news_df.isnull().sum())

id                   0
title                0
description          0
published_date       0
breaking_news        0
blob_image        1000
source_url           0
created_at           0
updated_at           0
published_at        35
created_by_id      382
updated_by_id       35
shares               0
comment_count        0
type                 0
dtype: int64


In [414]:
news_df.drop(columns=["blob_image", "shares", "comment_count", "type"], inplace=True)

In [415]:
news_df['published_at'] = news_df['published_at'].fillna(news_df['published_at'].mode()[0])

news_df['updated_by_id'] = news_df['updated_by_id'].fillna(news_df['updated_by_id'].mode()[0])

news_df['created_by_id'] = news_df['created_by_id'].fillna(news_df['created_by_id'].mode()[0])

In [416]:
print(news_df.isnull().sum())

id                0
title             0
description       0
published_date    0
breaking_news     0
source_url        0
created_at        0
updated_at        0
published_at      0
created_by_id     0
updated_by_id     0
dtype: int64


In [417]:
news_df['content'] = news_df['title'] + " " + news_df['description']

In [418]:
sinhala_stop_words = [
    "අය", "අතර", "ඉන්", "එක", "එය", "ඔබ", "ඔයා", "ඔහු", "ඔවුන්",
    "ඕනෑ", "ආයුබෝවන්", "ආදිය", "ඇයි", "ඇතුලත", "ඇත", "ඉන්පසු",
    "ඉස්සර", "එක්", "ඔය", "ඔබට", "ඔයාලා", "ඔයාව", "උදාහරණ", "උපුටා",
    "ඉදිරියට","එදා", "ඒ", "ඒවා", "ඕන", "ඔක්කෝම", "ඔවුන්",
    "අප", "අපේ", "ඇතුළත්", "ඇත", "ඉතා", "ඊයේ", "ඔබට", "ඔබගේ", "එහි",
    "එන්න","ඕක", "ආයුබෝවන්", "ඇත", "උසස්", "ඉහළ", "ඔබේ", "අය", "ඔව්",
    "එක්", "අපේ", "අද", "ඉන්", "ඉතා", "ඉතාම", "ඉහත", "අවශ්‍ය", "ඔයාලට"
]

In [419]:
def preprocess_sinhala_text(text):
    text = str(text)
    text = re.sub(r'[^\u0D80-\u0DFF\s]', '', text)
    text = text.strip()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in sinhala_stop_words]
    return " ".join(tokens)

In [421]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [422]:
news_df['content'] = news_df['content'].apply(preprocess_sinhala_text)

In [423]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(news_df['content'])

In [424]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [425]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=news_df['id'], columns=news_df['id'])

In [426]:
with open('cbf_model_cosine_sim.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)

In [427]:
with open('cbf_model_tfidf.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

In [428]:
def recommend_content_based(news_id, cosine_sim=cosine_sim, top_n=5):
    if news_id not in news_df['id'].values:
        return []

    idx = news_df[news_df['id'] == news_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    recommended_indices = [x[0] for x in sim_scores]

    return news_df.iloc[recommended_indices]['id'].tolist()

In [429]:
recommended_articles = recommend_content_based(23996, top_n=5)
print(recommended_articles)


[23097, 23794, 23158, 23819, 23710]


## CF

In [431]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp311-cp311-linux_x86_64.whl size=2505170 sha256=0c17b8aeae73eaabfa61479e3c81bb92e4df7c9137c597463996aad5744a18da
  Stored in directory: /root/.cache/pip/wheels/2a/8f/6e/7e2899163e2d85d8266daab4aa1cdabec7a6c56f83c015b5af
Successfully built scikit-surprise


In [432]:
import pandas as pd
import numpy as np
import pickle
from sklearn.neighbors import NearestNeighbors
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
import datetime

In [434]:
rec_feedback_df = pd.read_csv('rec_feedback.csv')

users_df = pd.read_csv('users.csv')

In [435]:
rec_feedback_df.head()

,id,feedback_type,time_stamp,comment,user_id,item_id,created_at,updated_at,created_by_id,updated_by_id
0,49554,click,2025-02-13 10:03:47.440412,NaN,1182,23942,NaN,NaN,NaN,NaN
1,49553,read,2025-02-13 10:03:42.411734,NaN,1182,23942,NaN,NaN,NaN,NaN
2,49552,click,2025-02-13 10:01:54.947236,NaN,1058,23942,NaN,NaN,NaN,NaN
3,49550,click,2025-02-13 10:01:31.955569,NaN,1058,23930,NaN,NaN,NaN,NaN
4,49549,click,2025-02-13 10:01:10.809916,NaN,1058,23941,NaN,NaN,NaN,NaN


In [436]:
rec_feedback_df.dtypes

,0
id,int64
feedback_type,object
time_stamp,object
comment,float64
user_id,int64
item_id,int64
created_at,float64
updated_at,float64
created_by_id,float64
updated_by_id,float64


In [440]:
print(rec_feedback_df.isnull().sum())

id               0
feedback_type    0
time_stamp       0
user_id          0
item_id          0
dtype: int64


In [445]:
interaction_weights = {'click': 1, 'read': 2, 'like': 3}
rec_feedback_df['weight'] = rec_feedback_df['feedback_type'].map(interaction_weights)

In [446]:
rec_feedback_df['time_stamp'] = pd.to_datetime(rec_feedback_df['time_stamp'])

In [450]:
from datetime import datetime

current_time = datetime.now()
print(current_time)

2025-02-17 09:05:04.470340


In [451]:
current_time = datetime.now()
time_decay_factor = 0.9  # Adjust based on how fast news ages
rec_feedback_df['time_weight'] = rec_feedback_df['time_stamp'].apply(
    lambda x: time_decay_factor ** ((current_time - x).days)
)

In [452]:
rec_feedback_df['final_weight'] = rec_feedback_df['weight'] * rec_feedback_df['time_weight']

In [453]:
reader = Reader(rating_scale=(0, 5))
data = Dataset.load_from_df(rec_feedback_df[['user_id', 'item_id', 'final_weight']], reader)

In [454]:
trainset, testset = train_test_split(data, test_size=0.2)
svd = SVD()
svd.fit(trainset)

In [455]:
with open('collaborative_model.pkl', 'wb') as f:
    pickle.dump(svd, f)

In [456]:
def recommend_collaborative(user_id, model=svd, top_n=5):
    """Recommend top N articles for a given user using SVD."""
    if user_id not in rec_feedback_df['user_id'].values:
        print(f"User {user_id} not found. Returning trending articles.")
        return rec_feedback_df.groupby('item_id')['final_weight'].sum().sort_values(ascending=False).head(top_n).index.tolist()

    all_items = rec_feedback_df['item_id'].unique()
    predictions = [model.predict(user_id, item) for item in all_items]

    top_recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:top_n]

    return [pred.iid for pred in top_recommendations]

In [458]:
user_id = 2303
top_recommendations = recommend_collaborative(user_id, svd, top_n=5)
print("Top 5 Recommendations:", top_recommendations)

Top 5 Recommendations: [23803, 23763, 23939, 23941, 20102]


## Hybrid Model

In [513]:
with open('cbf_model_cosine_sim.pkl', 'rb') as f:
    cosine_sim = pickle.load(f)

with open('cbf_model_tfidf.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('collaborative_model.pkl', 'rb') as f:
    knn = pickle.load(f)

In [514]:
# Content-Based Filtering
def recommend_content_based(news_id, cosine_sim=cosine_sim, top_n=5):
    if news_id not in news_df['id'].values:
        print(f"News ID {news_id} not found.")
        return []

    idx = news_df[news_df['id'] == news_id].index[0]
    sim_scores = sorted(enumerate(cosine_sim[idx]), key=lambda x: x[1], reverse=True)
    recommended_indices = [x[0] for x in sim_scores[1:top_n + 1]]

    return news_df.iloc[recommended_indices]['id'].tolist()

In [515]:
# Collaborative Filtering
def recommend_collaborative(user_id, user_item_matrix=user_item_matrix, knn=knn, top_n=5):
    if user_id not in user_item_matrix.index:
        print(f"User {user_id} not found. Returning top items.")
        return user_item_matrix.sum(axis=0).sort_values(ascending=False).head(top_n).index.tolist()

    user_index = user_item_matrix.index.get_loc(user_id)
    distances, indices = knn.kneighbors(user_item_matrix.iloc[user_index, :].values.reshape(1, -1), n_neighbors=top_n + 1)
    similar_users = user_item_matrix.index[indices.flatten()[1:]].tolist()
    recommended_items = rec_feedback_df[rec_feedback_df['user_id'].isin(similar_users)]['item_id'].value_counts().index.tolist()

    return recommended_items[:top_n]

In [516]:
# Hybrid Filtering
def recommend_hybrid(user_id, news_id, alpha=0.5, top_n=5):
    cbf_recommendations = recommend_content_based(news_id, top_n=top_n)
    cf_recommendations = recommend_collaborative(user_id, top_n=top_n)

    hybrid_scores = {}
    for item in set(cbf_recommendations + cf_recommendations):
        cbf_score = 1 if item in cbf_recommendations else 0
        cf_score = 1 if item in cf_recommendations else 0
        hybrid_scores[item] = alpha * cbf_score + (1 - alpha) * cf_score

    return sorted(hybrid_scores.keys(), key=lambda x: hybrid_scores[x], reverse=True)[:top_n]

In [518]:
with open('hybrid_recommendation_model.pkl', 'wb') as f:
    pickle.dump(recommend_hybrid, f)

In [519]:
user_id = 2303
news_id = 23996
top_n = 5

recommendations = recommend_hybrid(user_id, news_id, alpha=0.5, top_n=top_n)
print("Top Hybrid Recommendations:", recommendations)

AttributeError: 'csr_matrix' object has no attribute 'index'